<h1>Проект для "Викишоп"</h1>

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. Следует создать модель, которая будет классифицировать комментарии на позитивные и негативные.

**Описание данных**

 В вашем распоряжении набор данных с разметкой о токсичности правок. Столбец text в нём содержит текст комментария, а toxic — целевой признак.

**План работы**: первым делом подготовим полученные нами данные - приведем их в тот вид, с которым сможет работать потенциальная модель, затем - обучим модель, проверим качество (метрика F1 должна иметь значение не менее 0.75), и наконец - сделаем выводы.

<h3>Оглавление</h3>

1. [Шаг 1: Подготовка данных](#samples)
2. [Шаг 2: Обучение](#models)
3. [Шаг 3: Выводы](#test_sanity)

<h3>Шаг 1: Подготовка данных</h3>
<a id='samples'></a>

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from nltk.stem import WordNetLemmatizer 
import matplotlib.pyplot as plt
import re

In [2]:
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Ознакомимся с нашими данными.

Загрузим данные, посмотрим на первые пять строк, а также на общую информацию.

In [3]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [4]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Все выглядит так, как должно выглядеть. Пропусков нет.

*Лемматизация*

Лемматизируем текст.

In [6]:
lemmatizer = WordNetLemmatizer()

In [7]:
def lemmat(text):
    word_list = nltk.word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(w) for w in word_list])

In [8]:
data['lemmatzd'] = data['text'].apply(lemmat)

Проверим, что вышло.

In [9]:
data['lemmatzd'][0]

"Explanation Why the edits made under my username Hardcore Metallica Fan were reverted ? They were n't vandalism , just closure on some GAs after I voted at New York Dolls FAC . And please do n't remove the template from the talk page since I 'm retired now.89.205.38.27"

Ну, неплохо.

Немного почистим текст.

In [10]:
def clear_text(text):
    temp = re.sub(r'[^a-zA-Z]', ' ', text)
    return ' '.join(temp.split())

In [11]:
data['clear'] = data['lemmatzd'].apply(clear_text)

*TF-IDF*

Теперь применим TF-IDF.

In [12]:
stopwordss = set(stopwords.words('english'))
tfidf = TfidfVectorizer(stop_words=stopwordss)

А для дальнейшей работы нам потребуется разделить нашу выборку на обучающую и тестовую. Соотношение - 75:25. Валидационной не будет, поскольку будем использовать кросс-валидацию.

In [13]:
train, test = train_test_split(data, random_state=12, test_size=0.25)

Проведем преобразование, а также выделим столбец с целевым признаком.

In [14]:
corpus = tfidf.fit_transform(train['clear'])
train_Y = train['toxic']

test_X = tfidf.transform(test['clear'])
test_Y = test['toxic']

Готово.

**Вывод по Шагу 1**: итак, на данном шаге мы проделали важную работу - мы привели наши данные к тому виду, с которым могут работать модели - мы лемматизировали тексты, оставили в нем только необходимое, а также применили формулу TF-IDF. Получили набор признаков пригодный для дальнейшей работы.

<h3>Шаг 2: Обучение</h3>
<a id='models'></a>

Перед нами стоит задача классификации. Метрика - F1 (и она должна иметь значение не меньше 0.75). 

Попробуем в данном разделе логистическую регрессию, случайный лес и градиентный бустинг.  Также будем подбирать гиперпараметры.

Будем подбирать гиперпараметры RandomizedSearchCV и GridSearchCV.

**Логистическая регрессия**

In [15]:
params1 = {'class_weight': ['balanced', None], 'random_state':[12]}
model1 = LogisticRegression()
gs1 = GridSearchCV(model1, params1, cv=5, n_jobs=-1, scoring='f1')
gs1.fit(corpus, train_Y)

C:\Users\Acer\anaconda3\envs\practicum\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'class_weight': ['balanced', None],
                         'random_state': [12]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=0)

Подобрали лучшие гиперпараметры.

In [16]:
best1 = gs1.best_estimator_

Посмотрим на качество данной модели.

In [17]:
'Лучшее значение F1 данной модели на кросс-валидации: {:.2}'.format(gs1.best_score_)

'Лучшее значение F1 данной модели на кросс-валидации: 0.74'

Получили неплохое значение. Идем дальше.

**Случайный лес**

 RandomizedSearch.

In [18]:
#params2 = {'max_depth' : [5, 10, 15, 20, 30, 40],
#          'n_estimators' : [100, 200],
#          'criterion': ['gini', 'entropy']}
#model2 = RandomForestClassifier()
#gs2 = RandomizedSearchCV(model2, params2, n_iter=8, cv=5, n_jobs=-1, scoring='f1')
#gs2.fit(corpus, train_Y)

Посмотрим на лучшие параметры.

In [19]:
#gs2.best_params_
best_params = {'n_estimators': 100, 'max_depth': 40, 'criterion': 'gini'}

In [20]:
best2 = RandomForestClassifier(n_estimators=100, max_depth=40, criterion='gini')
best2.fit(corpus, train_Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=40, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Посмотрим на качество такой модели.

In [21]:
'Значение F1 данной модели на обучающей выборке: {:.2}'.format(f1_score(best2.predict(corpus), train_Y))

'Значение F1 данной модели на обучающей выборке: 0.046'

Получилось совсем грустно, видимо, данная модель не подходит для задачи.

**Градиентный бустинг**

Будем менять значения параметров максимальной глубины. 

In [22]:
model3 = LGBMClassifier()
params3 = {'max_depth' : [5, 10, 15, 20, 30, 40],
          'n_estimators' : [100]}
gs3 = GridSearchCV(model3, params3, cv=5, n_jobs=-1)
gs3.fit(corpus, train_Y)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=100,
                                      n_jobs=-1, num_leaves=31, objective=None,
                                      random_state=None, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15, 20, 30, 40],
                       

Подобрали лучшие гиперпараметры. 

In [23]:
best3 = gs3.best_estimator_

Глянем на качество.

In [24]:
'Лучшее значение F1 данной модели на кросс-валидации: {:.2}'.format(gs3.best_score_)

'Лучшее значение F1 данной модели на кросс-валидации: 0.96'

Очевидно - градиентный бустинг - наш вариант. Уточним его гиперпараметры.

In [25]:
gs3.best_params_

{'max_depth': 30, 'n_estimators': 100}

**Вывод по Шагу 2**: итак, на данном шаге мы подобрали лучшую модель, которая соответствует требованиям качества - метрика F1 ее на валидационной выборке не менее 0.75 (а даже 0.96), на следующем шаге мы проверим качество модели на тестовой выборке и убедимся, что выбрали хороший вариант.

<h3>Шаг 3: Выводы</h3>
<a id='test_sanity'></a>

Итак, на прошлом шаге мы выбрали лучшую модель. Это - градиентный бустинг. Проверим качество на тестовой выборке.

In [26]:
best_test = f1_score(best3.predict(test_X), test_Y)
'Значение F1 лучшей модели на тестовой выборке: {:.3}'.format(best_test)

'Значение F1 лучшей модели на тестовой выборке: 0.754'

Получили значение 0.754 . Это соответствует требованию. 

**Вывод по Шагу 3**: итак, мы убедились, что выбранная нами модель как лучшая действительно соответствует ожиданиям и показывает хорошие результаты - соответствующие требованию. Модель для данной задачи готова.